# **0. Preparation**

In order to train a Yolo v3 model using the previously generated dataset, we need to clone and compile darknet. After cloning and compiling darknet, we will import the card dataset we previously generated.

Darknet is an open source neural network framework used to train Yolo. As for Yolo, it is a neural network algorithm which works this way. The network divides the image into regions and predicts bounding boxes and probabilities for each region. Then these boxes are weighted by the predicted probabilities.



### **Clone darknet**

In [0]:
!git clone https://github.com/AlexeyAB/darknet

### **Compile darknet**

In [0]:
%cd darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
!cat Makefile
!make

### **Download dataset from Drive**


In [0]:
import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
  
DOWNLOAD_LOCATION = '/content/darknet/data/'
DRIVE_DATASET_FILE = '/content/gdrive/My Drive/ml/card-dataset.zip'

shutil.copy(DRIVE_DATASET_FILE, DOWNLOAD_LOCATION)

print('Successfully downloaded the dataset')

### **Unzip the dataset**

In [0]:
!unzip data/card-dataset.zip -d data/

# **1. Training the model**

With all the preparations already done, it's time to start the training. To do that, we will download the initial weights so we can start the process.

In [0]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

## **Set Yolo configurations**

We need to set configurations for Yolo in order to properly train. There are few settings which we need to change in the default yolov3.cfg file.

- batch
- subdivisions (if you get memory out error, increase this 16, 32 or 64)
- max_batches (it should be classes*2000)
- steps (it should be 80%, 90% of max_batches)
- classes (the number of classes which you are going to train)
- filters (the value for filters can be calculated using (classes + 5)x3 )


- classes = 52
- filters = (52 + 5) * 3 = 171
- max_batches = 52 * 2000 = 104000
- steps = (0.8 * 104000), (0.9 * 104000) = (83200, 93600)

In [0]:
!sed -i 's/batch=1/batch=64/g' cfg/yolov3.cfg
!sed -i 's/subdivisions=1/subdivisions=32/g' cfg/yolov3.cfg
!sed -i 's/max_batches = 500200/max_batches = 104000/g' cfg/yolov3.cfg
!sed -i 's/steps=400000,450000/steps=83200,93600/g' cfg/yolov3.cfg
!sed -i 's/classes=80/classes=52/g' cfg/yolov3.cfg
!sed -i 's/filters=255/filters=171/g' cfg/yolov3.cfg
!cat cfg/yolov3.cfg

### **Import pre-trained weights from drive (import pre-trained model)**

In [0]:
import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
  
BACKUP_FOLDER = '/content/darknet/backup'
DRIVE_YOLO_BACKUP = '/content/gdrive/My Drive/ml/yolov3_last_pretrained.weights'

shutil.copy(DRIVE_YOLO_BACKUP, BACKUP_FOLDER)

print('Successfully fetched the pretrained files for Yolo from Google drive')

### **Start the training**

In [0]:
# use the line below to train a fresh model
!./darknet detector train data/obj.data cfg/yolov3.cfg darknet53.conv.74

# use the line below to retrain your previous saved weight
#!./darknet detector train data/obj.data cfg/yolov3.cfg backup/yolov3_last.weights

### **Save trained weights**

In [0]:
# Once you have trained your model, you can save them to your Google drive. So that next time, you don't need to retrain
import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
  
YOLO_BACKUP = '/content/darknet/backup/yolov3_last.weights' #adjust the backup file name or keep it default
DRIVE_DIR = '/content/gdrive/My Drive/ml/' #adjust path in your Google drive, or keep it default

shutil.copy(YOLO_BACKUP, DRIVE_DIR)

print('Saved training data to drive at: ' + DRIVE_DIR)

# **2. Testing the model**

## **Running predictions**

In [0]:
#test the model on a picture
!./darknet detector test data/obj.data cfg/yolov3.cfg backup/yolov3_last_pretrained.weights /content/test.jpg

## **Display result**

In [0]:
def display_image(file_path = '/content/darknet/predictions.jpg'):
    import cv2
    import matplotlib.pyplot as plt
    import os.path

    if os.path.exists(file_path):
      img = cv2.imread(file_path)
      show_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
      plt.imshow(show_img)
    else:
      print('failed to open file')
    
display_image()